In [1]:
import torch
import numpy as np
import pandas as pd
import helper_functions as hf
from dataloader import JPXData
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataloader import default_collate
from torchvision import transforms
from statistics import mean
from tqdm import tqdm
import itertools

# pd.set_option('display.max_rows', 100)
torch.set_printoptions(profile="default")
device = "cuda" if torch.cuda.is_available() else "cpu"

/home/aymane/Kaggle/jpx-2022/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Run this to create the new modified_data folder
# ~ 1 min runtime

# hf.concat_data()

# Run this to tokenize Section/Product and NewMarketSegmentColumns on stock_list.csv
# ~ 20 secs runtime

# hf.tokenize_stock_list()

# Run this to adjust the closing price
# ~ 2 mins runtime

# hf.adjust_price()

In [3]:
stock_prices = pd.read_csv("modified_data/autoencoder_data.csv")

In [4]:
# ~ 3 month testing time frame 2022-01-04 to 2022-04-28
train_data = stock_prices[stock_prices["Date"] < '2022-01-04'].copy().reset_index(drop=True)
test_data  = stock_prices[stock_prices["Date"] >= '2022-01-04'].copy().reset_index(drop=True)

In [5]:
train_class = JPXData(data=train_data)
trainloader = torch.utils.data.DataLoader(train_class, batch_size=1, shuffle=True, num_workers=5)
test_class  = JPXData(data=test_data, train=False)
testloader  = torch.utils.data.DataLoader(JPXData(data=test_data), batch_size=1, shuffle=False, num_workers=5)

/home/aymane/Kaggle/jpx-2022/dataloader.py:148: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  means_stds = torch.tensor([np.array(calc_mean_std(code)) for code in self.stock_id.tolist()],dtype=torch.float)
/home/aymane/Kaggle/jpx-2022/dataloader.py:100: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  data = np.array([calc_io(date) for date in self.dates])
/home/aymane/Kaggle

In [6]:
train_class[1220]

(array([-1.0993493 , -0.00413453,  0.06098191, ...,  0.02222464,
        -0.28374362, -0.08658674], dtype=float32),
 0.0061527328,
 ['2021-12-30'])

In [7]:
# Ranking Securities Codes by rate of change from t+1 to t+2
stock_prices = pd.read_csv("modified_data/stock_prices.csv")
date = "2017-01-04"
targets = stock_prices.query("Date == @date")[["Target", "SecuritiesCode"]].fillna(0)
df = targets.sort_values(by="Target")
df

,Target,SecuritiesCode
1733,-0.069874,9474
1856,-0.067387,9983
1136,-0.062112,6875
1852,-0.059276,9974
1078,-0.057514,6707
...,...,...
1695,0.094378,9233
404,0.098553,3547
1000,0.104668,6425
1297,0.106332,7600


In [8]:
a = train_class
print(len(a[0][0]))

138000


In [9]:
# # Verifying if the ranking is the same
# securities_codes    = stock_prices["SecuritiesCode"]
# securities_list     = sorted(list(set(securities_codes)))
# securities_dict     = {i: securities_list.index(i) for i in securities_list}

# a = dataloader[0][1]
# sorted_args = torch.argsort(a, descending=True)
# securities_sort = [list(securities_dict.keys())[list(securities_dict.values()).index(i)] for i in sorted_args]
# print(securities_sort)

In [23]:
69*2000

138000

In [19]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(138000, 7000),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(7000, 5000),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(5000, 2000),
            # nn.LeakyReLU(negative_slope=0.1),
            # nn.Linear(1000, 1500),
            # nn.LeakyReLU(negative_slope=0.1),
            # nn.Linear(1500, 2000),
        )

    def forward(self, x):
        output = self.layer_stack(x)
        return output

In [20]:
net = Net().cuda()

In [21]:
criterion = nn.MSELoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9)

In [22]:
loss_a1 = []

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels, date = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = hf.loss_function(outputs * 10, labels * 10, criterion=nn.HuberLoss(delta=5))
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(net.parameters(), 1.5)
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        loss_a1.append(running_loss)
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.10f}')
            running_loss = 0.0
        # torch.save(net.state_dict(), f'saved_models/{epoch}.pt')

print('Finished Training')

/home/aymane/Kaggle/jpx-2022/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:981: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 2000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)


RuntimeError: CUDA out of memory. Tried to allocate 3.60 GiB (GPU 0; 5.80 GiB total capacity; 3.99 GiB already allocated; 540.12 MiB free; 4.01 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
plt.plot(loss_a1)
plt.show()

In [ ]:
pred_df = pd.DataFrame()
dates = []
securities = []
preds = []
targets = []

stock_ids = test_class.stock_id.tolist()

for X, target, date in tqdm(test_class):
    X = X.cuda()
    pred = net(X)

    target_list = target.tolist()

    dates.append(date*2000)
    securities.append(stock_ids)
    preds.append(pred.tolist())
    targets.append(target_list)

    new_dates = list(itertools.chain.from_iterable(dates))
    new_securities = list(itertools.chain.from_iterable(securities))
    new_preds = list(itertools.chain.from_iterable(preds))
    new_targets = list(itertools.chain.from_iterable(targets))

pred_df["Date"] = new_dates
pred_df["SecuritiesCode"] = new_securities
pred_df["Prediction"] = new_preds
pred_df["Target"] = new_targets
print(pred_df)
# predictions = pd.DataFrame(list(zip(dates, securitiescodes, targets)), columns=["Date", "SecuritiesCode", "Target"]

In [ ]:
dates = pred_df.groupby("Date")

securities_dict = {i: stock_ids.index(i) for i in stock_ids}
rank = []
for i in tqdm(dates):
    sorted_args = torch.argsort(torch.tensor(list(i[1]["Prediction"])), descending=True)
    securities_sort = [list(securities_dict.keys())[list(securities_dict.values()).index(i)] for i in sorted_args]
    sorted_ranks = [securities_sort.index(i) for i in stock_ids]
    rank.append(sorted_ranks)

new_rank = list(itertools.chain.from_iterable(rank))
pred_df["Rank"] = new_rank

In [ ]:
hf.calc_spread_return_sharpe(pred_df)

In [ ]:
pred_df